<a href="https://colab.research.google.com/github/movi-data/painel_performance/blob/main/painel_performance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Relatório Plataformas

In [ ]:
import pandas as pd

# --- ETAPA 1: Carregar o arquivo ---
df_propostas = pd.read_csv("propostas.csv", sep=";", encoding="utf-8", low_memory=False)

# --- ETAPA 2: Preparar Colunas ---

# Tranformar 'Canal' em maiuscula
df_propostas['Canal'] = df_propostas['Canal'].str.upper()
# Filtrar o primeiro Canal na coluna 'Canal'
df_propostas['Canal'] = df_propostas['Canal'].str.split(',').str[0]

# --- ETAPA 3: Fazer agrupamento e ordernar ---

# Agrupar leads por mês na coluna 'Data' e 'Canal'
leads_por_canal_mes = df_propostas.groupby([pd.to_datetime(df_propostas["Data"], format="%d/%m/%Y", errors='coerce').dt.to_period("d"),
                                            "Canal"]).size().reset_index(name="Quantidade de Leads")
# Ordenar por 'Quantidade de Leads'
leads_por_canal_mes = leads_por_canal_mes.sort_values(by="Quantidade de Leads", ascending=False)

# --- ETAPA 4: Criar arquivo csv e download ---

# Criar arquivo .csv
leads_por_canal_mes.to_csv("leads_por_canal_mes.csv",sep=";" ,index=False)

# Download do arquivo
from google.colab import files
files.download("leads_por_canal_mes.csv")

# Vizualização para conferência
leads_por_canal_mes.head()

,Data,Canal,Quantidade de Leads
115,2025-06-11,INSTAGRAM,3013
1207,2025-09-10,INSTAGRAM,2342
148,2025-06-14,INSTAGRAM,2289
910,2025-08-16,WHATSAPP,2264
1195,2025-09-09,INSTAGRAM,2252
